<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto:2</font></h1>

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup=BeautifulSoup(source,'xml')

In [4]:
table1=soup.find('table')

In [5]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
column_name=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_name)

In [6]:
table=soup.find('table')

<h5>Getting all values in tr and seperating each td within by ","</h5>

In [7]:
# Search all the postcode, borough, neighborhood 
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [8]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h5><font size=5> Data Cleaning and exploring</font></h5>

<h5>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</h5>

In [9]:
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</h5>

In [10]:
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>rows with same postalcode have to be combined into one row seaparated by a comma.</h5>

In [11]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()
df_new.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h5>Using the .shape method to print the number of rows of the new dataframe.</h5>

In [12]:
df_new.shape

(103, 3)

<h1>Question 2: Get the latitude and the longitude coordinates of each neighborhood. </h1>

<h3>Using the csv file of the coordinates</h3>

In [13]:
df_con=pd.read_csv(r"http://cocl.us/Geospatial_data")
df_con.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_con.rename(columns={'Postal Code':'Postalcode'},inplace=True)
df_con.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_merge=pd.merge(df_con,df,on='Postalcode')
df_merge.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [22]:
df_final=df_merge[["Postalcode","Borough","Neighborhood","Latitude","Longitude"]]
df_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
